In [1]:
import os

import numpy as np
# import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use([
  "dark_background",
  # "https://raw.githubusercontent.com/cfrontin/tools_cvf/main/tools_cvf/stylesheet_cvf.mplstyle",
  # "https://raw.githubusercontent.com/cfrontin/tools_cvf/main/tools_cvf/stylesheet_nrel.mplstyle",
])

import weis.visualization.utils as viz_toolbox
import weis.visualization.opt_plotting as opt_plotting

# Example 17: IEA22 `OpenFAST` Controls Optimization

In this example, we can optimize a controller for a semisubmersible floating offshore wind turbine (FOWT) platform based around the IEA 22MW reference turbine.
We will consider optimizations using the following optimizers:
- COBYLA optimizer (derivative-free)
- SLSQP optimizer (gradient-based), and
- differential evolution (DE) (an evolutionary algorithm)

## Metadata loading

In the following code sections we will set up the loading of the metadata files.

In [2]:
# standard locations of output dirs based on template for ex. 17
dir_template = "17_IEA22_OptStudies/of_ROSCO_%s"
dir_COBYLA = dir_template % "COBYLA"
dir_SLSQP = dir_template % "SLSQP"
dir_DE = dir_template % "DE"

# OM optimization log database files
fn_log_COBYLA = os.path.join(dir_COBYLA, "log_opt.sql")
fn_log_SLSQP = os.path.join(dir_SLSQP, "log_opt.sql")
fn_log_DE = os.path.join(dir_DE, "log_opt.sql_%s")

# WEIS stashes design/constraint/objective var files located here
fn_vars_COBYLA = os.path.join(dir_COBYLA, "problem_vars.yaml")
fn_vars_SLSQP = os.path.join(dir_SLSQP, "problem_vars.yaml")
fn_vars_DE = os.path.join(dir_DE, "problem_vars.yaml")

In [3]:
# ... load the variables files
do_COBYLA = True
do_SLSQP = False
do_DE = False
unification_list = []
feas_tol=1e-4

# cost approximation variables
N_DV = 8
DLC_map = {
  "1.1": {
    "N_var": 3,
    "N_seed": 1,
  },
}
P_map = {
  "COBYLA": 1,
  "SLSQP": 2*N_DV,
  "DE": 5*N_DV,
}
N_CPU = 104

if do_COBYLA:
  vars_COBYLA = viz_toolbox.load_vars_file(fn_vars_COBYLA)
  unification_list.append(vars_COBYLA)
if do_SLSQP:
  vars_SLSQP = viz_toolbox.load_vars_file(fn_vars_SLSQP)
  unification_list.append(vars_SLSQP)
if do_DE:
  # vars_DE = viz_toolbox.load_vars_file(fn_vars_DE)
  vars_DE = viz_toolbox.load_vars_file(fn_vars_SLSQP)  # DEBUG!!!!!
  unification_list.append(vars_DE)
# this call verifies, (optionally) unifies, and corrects the var files
vars_unified = viz_toolbox.verify_vars(*unification_list)

invalid escape sequence '\s'

FileNotFoundError: [Errno 2] No such file or directory: '17_IEA22_OptStudies/of_ROSCO_COBYLA/problem_vars.yaml'

## Data loading

With the metadata loaded, we can now load the primary data from the various methods.
The COBYLA and SLSQP data is loaded first, with a simple serial loader, which are used because these methods either run in a serial fashion (with F.D. derivatives calculated in parallel in the case of SLSQP).
The DE data, since it is run in parallel, is loaded using a parallel data loader.

After the data is loaded, we show any differences in the keys found between the COBYLA/SLSQP methods and pretty-print the variables with icons representing whether they are objective functions (`**`), design variables (`--`), constraints (`<>`), or other (`??`).

In [ ]:
# load the data from the OM DB
if do_COBYLA:
  # dataOM_COBYLA = viz_toolbox.load_OMsql(fn_log_COBYLA)
  dataOM_COBYLA = viz_toolbox.load_OMsql(fn_log_COBYLA, parse_multi=True)
if do_SLSQP:
  # dataOM_SLSQP = viz_toolbox.load_OMsql(fn_log_SLSQP)
  dataOM_SLSQP = viz_toolbox.load_OMsql(fn_log_SLSQP, parse_multi=True)
if do_DE:
  dataOMmulti_DE = viz_toolbox.load_OMsql_multi(
    fn_log_DE % "*",
    meta_in=fn_log_DE % "meta",
  )
  dataOMbest_DE = viz_toolbox.consolidate_multi(
    dataOMmulti_DE,
    vars_DE,
  )

# describe the keys that have been found
print()
keys_all, _, _ = viz_toolbox.compare_om_data(
  dataOM_COBYLA,
  # dataOM_SLSQP,
  # dataOM_SLSQP,
  dataOM_COBYLA,
  # "COBYLA", "SLSQP",
  # "SLSQP2", "SLSQP",
  "COBYLA", "COBYLA2",
  verbose=True,
)
print()

# grab the keys that we have in the unified vars
keys_obj = [v["name"] for k, v in vars_unified["objectives"].items()]
keys_DV = [v["name"] for k, v in vars_unified["design_vars"].items()]
keys_constr = {v["name"]: [v["lower"], v["upper"]] for k, v in vars_unified["constraints"].items()}

# pretty print the case we're looking at
viz_toolbox.prettyprint_variables(keys_all, keys_obj, keys_DV, keys_constr)

## Feasibility pre-processing

Now, we will can grab and evaluate the feasibility of the DE iterations across all the ranks.

In [ ]:
if do_DE:
  # extract and install feasibility evaluations
  feas, vfeas = viz_toolbox.get_feasible_iterations(
    dataOMmulti_DE, vars_unified,
    feas_tol=feas_tol,
  )
  dataOMmulti_DE["feas_total"] = feas
  for k, v in vfeas.items():
    dataOMmulti_DE[f"feas_{k}"] = v

## Plotting

### Differential Evolution results

First, we can examine the results of the DE optimization.
At each of 100 iterations, there are 104 processors working the problem.
The figure shows the progression of the optimization with feasible simulations in green, infeasible in red, the iteration-wise best result in cyan, and the value of the discovered minimizer in yellow dashes.

In the following figure, we show the iteration-over-iteration convergence of the iteration-wise best feasible estimate toward the discovered minimizer, which demonstrates regular convergence toward this value.

In [ ]:
if do_DE:
  # plot DE results
  var_of_interest = keys_obj[0]
  fig, ax = plt.subplots()
  ax.scatter([], [], s=3.0, c="g", label="feasible sample")
  ax.scatter([], [], s=3.0, c="m", label="infeasible sample")
  ax.scatter(
    dataOMmulti_DE["iter"],
    dataOMmulti_DE[var_of_interest],
    s=3.0,
    c=["g" if d else "m" for d in dataOMmulti_DE["feas_total"]],
    alpha=0.5,
    label="_simulation iterations_",
  )
  ax.plot(
    range(np.max(dataOMmulti_DE["iter"])+1),
    [
      np.min(np.array(dataOMmulti_DE[var_of_interest])[
        dataOMmulti_DE["feas_total"].flatten() & (np.array(dataOMmulti_DE["iter"]) == iter).flatten()
      ]) if len(
        np.array(dataOMmulti_DE[var_of_interest])[
          dataOMmulti_DE["feas_total"].flatten() & (np.array(dataOMmulti_DE["iter"]) == iter).flatten()
        ]
      ) else np.inf for iter in range(np.max(dataOMmulti_DE["iter"])+1)
    ],
    c="c",
    zorder=1000,
    label="best feasible estimate",
  )
  ax.plot(
    range(np.max(dataOMmulti_DE["iter"])+1),
    (np.min(
      np.array(dataOMmulti_DE[var_of_interest])[dataOMmulti_DE["feas_total"].flatten()]
    ) if len(
      np.array(dataOMmulti_DE[var_of_interest])[dataOMmulti_DE["feas_total"].flatten()]
    ) else np.inf)*np.ones_like(range(np.max(dataOMmulti_DE["iter"])+1)),
    "--y",
    zorder=500,
    label="discovered minimizer",
  )
  ax.grid(which="major", alpha=0.25)
  ax.set_xlabel("iteration number")
  ax.set_ylabel("damage equivalent load `TwrBsMyt` (N???)")
  ax.legend()

In [ ]:
if do_DE:
  print(f"discovered minimizer: {(np.min(
        np.array(dataOMmulti_DE[keys_obj[0]])[dataOMmulti_DE["feas_total"].flatten()]
      ) if len(
        np.array(dataOMmulti_DE[keys_obj[0]])[dataOMmulti_DE["feas_total"].flatten()]
      ) else np.inf)}")

In [ ]:
if do_DE:
  # plot DE results
  for var_of_interest, title_VoI in zip(
    [
      "tune_rosco_ivc.zeta_pc:0", "tune_rosco_ivc.zeta_pc:1", "tune_rosco_ivc.zeta_pc:2",
      "tune_rosco_ivc.omega_pc:0", "tune_rosco_ivc.omega_pc:1", "tune_rosco_ivc.omega_pc:2",
      "tune_rosco_ivc.Kp_float", "tune_rosco_ivc.ptfm_freq",
    ],
    [
      "tune_rosco_ivc.zeta_pc:0", "tune_rosco_ivc.zeta_pc:1", "tune_rosco_ivc.zeta_pc:2",
      "tune_rosco_ivc.omega_pc:0", "tune_rosco_ivc.omega_pc:1", "tune_rosco_ivc.omega_pc:2",
      "tune_rosco_ivc.Kp_float", "tune_rosco_ivc.ptfm_freq",
    ],
  ):
    dim = 0 if len(var_of_interest.split(':')) == 1 else int(var_of_interest.split(':')[-1])
    var_of_interest = var_of_interest.split(':')[0]
    fig, ax = plt.subplots()
    ax.scatter([], [], s=3.0, c="g", label="feasible sample")
    ax.scatter([], [], s=3.0, c="m", label="infeasible sample")
    var_to_plot = np.array(dataOMmulti_DE[var_of_interest]).reshape(len(dataOMmulti_DE["iter"]), -1)
    ax.scatter(
      dataOMmulti_DE["iter"],
      var_to_plot[:,dim],
      s=3.0,
      c=["g" if d else "m" for d in dataOMmulti_DE["feas_total"]],
      alpha=0.5,
      label="_simulation iterations_",
    )
    if (var_of_interest in keys_constr) or (var_of_interest in keys_DV):
      var_type_key = "constraints" if (var_of_interest in keys_constr) else "design_vars"
      ax.plot(
        range(np.max(dataOMmulti_DE["iter"])+1),
        vars_unified[var_type_key][var_of_interest]["lower"]*np.ones_like(range(np.max(dataOMmulti_DE["iter"])+1)),
        "b:",
        label="lower bound",
      )
      ax.plot(
        range(np.max(dataOMmulti_DE["iter"])+1),
        vars_unified[var_type_key][var_of_interest]["upper"]*np.ones_like(range(np.max(dataOMmulti_DE["iter"])+1)),
        "r:",
        label="upper bound",
      )
      yll = None
      ylh = None
      if not np.isinf(vars_unified[var_type_key][var_of_interest]["lower"]) and not np.isinf(vars_unified[var_type_key][var_of_interest]["upper"]):
        yll = vars_unified[var_type_key][var_of_interest]["lower"] - 0.25*(vars_unified[var_type_key][var_of_interest]["upper"] - vars_unified[var_type_key][var_of_interest]["lower"])
        ylh = vars_unified[var_type_key][var_of_interest]["upper"] + 0.25*(vars_unified[var_type_key][var_of_interest]["upper"] - vars_unified[var_type_key][var_of_interest]["lower"])
      else:
        if not np.isinf(vars_unified[var_type_key][var_of_interest]["lower"]):
          yll = vars_unified[var_type_key][var_of_interest]["lower"] - 0.25*np.abs(vars_unified[var_type_key][var_of_interest]["lower"])
        if not np.isinf(vars_unified[var_type_key][var_of_interest]["upper"]):
          ylh = vars_unified[var_type_key][var_of_interest]["upper"] + 0.25*np.abs(vars_unified[var_type_key][var_of_interest]["upper"])
      # ax.set_ylim(yll, ylh)
    ax.grid(which="major", alpha=0.25)
    ax.set_xlabel("iteration number")
    ax.set_ylabel(title_VoI)
    ax.legend()

In [ ]:
if do_DE:
  # plot DE results

  for var_of_interest, title_VoI in zip(
    ["aeroelastic.rotor_overspeed", "aeroelastic.avg_pitch_travel", "sse_tune.tune_rosco.PC_Ki", "sse_tune.tune_rosco.PC_Kp"],
    ["rotor overspeed (-)", "average pitch travel (rad/s)", "PC_Ki", "PC_Kp"],
  ):
    fig, ax = plt.subplots()
    ax.scatter([], [], s=3.0, c="g", label="feasible sample")
    ax.scatter([], [], s=3.0, c="m", label="infeasible sample")
    var_to_plot = np.array(dataOMmulti_DE[var_of_interest]).reshape(len(dataOMmulti_DE["iter"]), -1)
    for dim in range(var_to_plot.shape[-1]):
      ax.scatter(
        dataOMmulti_DE["iter"],
        var_to_plot[:,dim],
        s=3.0,
        c=["g" if d else "m" for d in dataOMmulti_DE["feas_total"]],
        alpha=0.5,
        label="_simulation iterations_",
      )
    if (var_of_interest in keys_constr) or (var_of_interest in keys_DV):
      var_type_key = "constraints" if (var_of_interest in keys_constr) else "design_vars"
      ax.plot(
        range(np.max(dataOMmulti_DE["iter"])+1),
        vars_unified[var_type_key][var_of_interest]["lower"]*np.ones_like(range(np.max(dataOMmulti_DE["iter"])+1)),
        "b:",
        label="lower bound",
      )
      ax.plot(
        range(np.max(dataOMmulti_DE["iter"])+1),
        vars_unified[var_type_key][var_of_interest]["upper"]*np.ones_like(range(np.max(dataOMmulti_DE["iter"])+1)),
        "r:",
        label="upper bound",
      )
      yll = None
      ylh = None
      if not np.isinf(vars_unified[var_type_key][var_of_interest]["lower"]) and not np.isinf(vars_unified[var_type_key][var_of_interest]["upper"]):
        yll = vars_unified[var_type_key][var_of_interest]["lower"] - 0.25*(vars_unified[var_type_key][var_of_interest]["upper"] - vars_unified[var_type_key][var_of_interest]["lower"])
        ylh = vars_unified[var_type_key][var_of_interest]["upper"] + 0.25*(vars_unified[var_type_key][var_of_interest]["upper"] - vars_unified[var_type_key][var_of_interest]["lower"])
      else:
        if not np.isinf(vars_unified[var_type_key][var_of_interest]["lower"]):
          yll = vars_unified[var_type_key][var_of_interest]["lower"] - 0.25*np.abs(vars_unified[var_type_key][var_of_interest]["lower"])
        if not np.isinf(vars_unified[var_type_key][var_of_interest]["upper"]):
          ylh = vars_unified[var_type_key][var_of_interest]["upper"] + 0.25*np.abs(vars_unified[var_type_key][var_of_interest]["upper"])
      # ax.set_ylim(yll, ylh)
    ax.grid(which="major", alpha=0.25)
    ax.set_xlabel("iteration number")
    ax.set_ylabel(title_VoI)
    ax.legend()

In [ ]:
if do_DE:
  # plot DE optimization convergence results
  var_of_interest = keys_obj[0]
  fig, ax = plt.subplots()
  ax.semilogy(
    range(np.max(dataOMmulti_DE["iter"])+1),
    np.abs([
      np.min(
        np.array(dataOMmulti_DE[var_of_interest])[
          dataOMmulti_DE["feas_total"].flatten() & (np.array(dataOMmulti_DE["iter"]) == iter).flatten()
        ]
      ) if len(
        np.array(dataOMmulti_DE[var_of_interest])[
          dataOMmulti_DE["feas_total"].flatten() & (np.array(dataOMmulti_DE["iter"]) == iter).flatten()
        ]
      ) else np.inf for iter in range(np.max(dataOMmulti_DE["iter"])+1)
    ]
    - (np.min(
      np.array(
        dataOMmulti_DE[var_of_interest]
      )[dataOMmulti_DE["feas_total"].flatten()]
    ) if len(
      np.array(
        dataOMmulti_DE[var_of_interest]
      )[dataOMmulti_DE["feas_total"].flatten()]
    ) else np.inf)*np.ones_like(range(np.max(dataOMmulti_DE["iter"])+1)))/(
      np.min(
        np.array(
          dataOMmulti_DE[var_of_interest]
        )[dataOMmulti_DE["feas_total"].flatten()]
      ) if len(
        np.array(
          dataOMmulti_DE[var_of_interest]
        )[dataOMmulti_DE["feas_total"].flatten()]
      ) else np.inf
    ),
    c="c",
    label="error in iteration-wise best feasible estimate",
  )
  ax.grid(which="major", alpha=0.25)
  ax.set_xlabel("iteration number")
  ax.set_ylabel(
    "apparent percent absolute error in "
    + "\ndamage equivalent load `TwrBsMyt` (%)"
  )

### Combined results

With the DE results in tow, we can now evaluate them with respect to the other solutions.
In the following plots, we will evaluate the optimization trajectories of the three optimizers.
In the first plot, the objective function for optimization is shown, and in the second, the design variables are shown.
Each of the markers is either filled for a feasible sample or unfilled for infeasible sample at a given iteration.
DE results are the best-available feasible instance at a given iteration, as shown above.

In [ ]:
opt_plotting.plot_conv(
  keys_obj,
  {
    "DE": (dataOMbest_DE, vars_DE),
    "SLSQP": (dataOM_SLSQP, vars_SLSQP),
    "COBYLA": (dataOM_COBYLA, vars_COBYLA),
  },
  feas_tol=feas_tol,
  alpha=0.5,
) ;

In [ ]:
opt_plotting.plot_conv(
  keys_DV,
  {
    "DE": (dataOMbest_DE, vars_DE),
    "SLSQP": (dataOM_SLSQP, vars_SLSQP),
    "COBYLA": (dataOM_COBYLA, vars_COBYLA),
  },
  feas_tol=feas_tol,
  alpha=0.5,
) ;

In the final subplot, below, we show the constraints active on the problem, which are numerous.
In this plot, filled (unfilled) markers represent feasibility (infeasibility) according to the constraint of interest on the displayed iteration.

In [ ]:
opt_plotting.plot_conv(
  keys_constr,
  {
    "DE": (dataOMbest_DE, vars_DE),
    "SLSQP": (dataOM_SLSQP, vars_SLSQP),
    "COBYLA": (dataOM_COBYLA, vars_COBYLA),
  },
  feas_tol=feas_tol,
  alpha=0.5,
  use_casewise_feasibility=True,
) ;

In [ ]:
fig_cost, ax_cost = plt.subplots()
fig_wallclock, ax_wallclock = plt.subplots()

for vars, dataOM, method_name in zip(
  [vars_COBYLA, vars_SLSQP, vars_DE],
  [dataOM_COBYLA, dataOM_SLSQP, dataOMbest_DE],
  ["COBYLA", "SLSQP", "DE"],
):
  obj_name = next(iter(vars["objectives"].keys()))
  obj_vals = dataOM[obj_name]
  iters = np.array(range(len(obj_vals)))
  M_iter = P_map[method_name]*np.sum([dlc["N_var"]*dlc["N_seed"] for dlc in DLC_map.values()])
  print(f"M_iter: {M_iter}")
  cost_coeff = iters*M_iter
  wallclock_coeff = iters*M_iter/min(N_CPU, M_iter)

  ax_cost.loglog(cost_coeff, np.abs(obj_vals - obj_vals[-1])/obj_vals[-1], label=method_name)
  ax_wallclock.loglog(wallclock_coeff, np.abs(obj_vals - obj_vals[-1])/obj_vals[-1], label=method_name)

ax_cost.set_xlabel("total solve-normalized cost, $C_{\\mathrm{total}}/T_{\\mathrm{solve}}$ (-)")
ax_wallclock.set_xlabel("solve-normalized wallclock time, $T_{\\mathrm{total}}/T_{\\mathrm{solve}}$ (-)")
for ax in [ax_cost, ax_wallclock]:
  ax.set_ylabel("normalized convergence w.r.t. terminal value (-)")
  ax.legend()